# test notebook for stella ZAMS calc code

In [12]:
# imports
import numpy as np
import pandas as pd
from scipy.interpolate import interpn

# constants
import constants as c
from scipy.constants import N_A

# plotting
import matplotlib.pyplot as plt
import seaborn as sb
sb.set_context("talk")
plt.rcParams['font.family'] = 'monospace'   # Fonts
plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'

In [6]:
import interpolate

interpolate.interp_k(1.1,1.1e6)

array([2.37514896])

Read Sections 18.0, 18.1, and 18.2 of Numerical Recipes, as it will be necessary
to implement the method of shooting to a fitting point (their shootf function) to
solve the boundary-value problem posed by the four coupled ordinary differential
equations of stellar structure and evolution. You will also need a program similar
to the newt function described in Section 9.7 of Numerical Recipes or in the
SciPy documentation to repeatedly call a shootf-like function, calculate updated
boundary values, and ultimately obtain a converged solution. Faster convergence
can usually be obtained by using fractions of the updates suggested by a newt-
like function. Ordinary differential equation (ODE) solvers like those described
in Sections 17.0, 17.1, and 17.2 of Numerical Recipes or the SciPy documentation
are also critical.

In [ ]:
import energy
import density

def load_inner(Pc, Tc):
    
    return (m, l, P, r, T)

def load_outer(L_star, R_star):
    
    return (m, l, P, r, T)

def del_rad(m, l, P, rho, T):
    """
    Calculate del_rad based on 4.30 in HKT
    """
    first_term = 3/(16*np.pi*c.a*c.c*c.G)
    kappa = interpolate.interp_k(rho, T)
    second_term = P*kappa*l/(m*T**4)
    return first_term*second_term

del_ad = 0.4 # assuming ideal gas, complete ionization

def odeez(m, l, P, r, T):
    rho = density.density(P,T) # density
    dldm = energy.energy_gen(rho,T) #change in Luminosity with enclosed mass
    dPdm = -c.G*m/(4*np.pi*r**4) # change in pressure with enclosed mass
    drdm = 1/(4*np.pi*r**2 * rho) # mass conservation eq.
    del_radiative = del_rad(m, l, P, rho, T)
    if del_radiative > del_ad:
        del_actual = del_ad
    else:
        del_actual = del_radiative
    dTdm = -c.G*m*T*del_actual/(4*np.pi*P*r**4) # change in temperature with enclosed mass
    return (dldm, dPdm, drdm, dTdm)
    
from scipy.integrate import solve_ivp
    
@np.vectorize
def shootingf(Mr_fit, Pc, Tc, L_star, R_star, n=50):
    Mr_cen, _, _, _, _ = load_inner(Pc, Tc)
    Mr_surf, _, _, _, _ = load_outer(L_star, R_star)
    Mr_in = np.linspace(Mr_cen, Mr_fit, num=n)
    Mr_out = np.linspace(Mr_surf, Mr_fit, num=n)
    
    sol_inner = solve_ivp(odeez, (Mr_in[0], Mr_in[-1]), load_inner(Pc, Tc), t_eval=Mr_in)
    
    sol_outer = solve_ivp(odeez, (Mr_out[0], Mr_out[-1]), load_outer(L_star, R_star), t_eval=Mr_out) 

    return sol_inner, sol_outer

from scipy import optimize
solution = optimize.newton(shootingf, )